In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.datasets import load_boston
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
dataset = load_breast_cancer()
data = dataset['data']
target = dataset['target']
X_train, X_test, y_train, y_test = \
    train_test_split(data, target, test_size=.25, random_state=42)
    

In [3]:
n = X_train.shape[1]
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)

In [4]:
X_train_scaled[0]

array([-0.3491385 , -1.4385134 , -0.41172594, -0.39047945, -1.86366224,
       -1.26860702, -0.8261705 , -0.95286584, -1.72936809, -0.9415409 ,
       -0.86971354, -1.35865343, -0.83481508, -0.57230675, -0.74586844,
       -0.65398318, -0.52583522, -0.94677144, -0.5378173 , -0.6344946 ,
       -0.54268485, -1.65565455, -0.58986402, -0.52555984, -1.51066923,
       -0.89149994, -0.75021714, -0.91671062, -0.92508584, -0.80841112], dtype=float32)

In [60]:
tf.reset_default_graph()

with tf.name_scope("input"):
    # define variables
    X = tf.placeholder(tf.float32, shape=[None, n], name='x')
    y = tf.placeholder(tf.float32, shape=[None, 1], name='y')

with tf.name_scope("regression"):
    # define variables
    W = tf.Variable(tf.truncated_normal([n,1], dtype=tf.float32, stddev=.05), name='weights')
    b = tf.Variable(tf.zeros([1], dtype=tf.float32), name='biases')
    
with tf.name_scope("operations"):
    # regular calculation of pred, similar to linear regression
    st1 = tf.add(tf.matmul(X ,W), b)
    
    # sigmoid converts from 0 to 1
    y_pred = tf.nn.sigmoid(st1)
        
    # compute regular error functions
    squared_error = tf.square(tf.subtract(y_pred, y))
    loss = tf.reduce_sum(squared_error)
    
    # result, needs to be 0 or 1
    res = tf.round(y_pred)
    
    # define optimization
    learning_rate = tf.placeholder(tf.float32)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss)


In [61]:
# tensorboard
# creat a summary for x and y
tf.summary.scalar("loss", loss)
summary_op = tf.summary.merge_all()

# no need to specify graph
writer = tf.summary.FileWriter('./example', graph=tf.get_default_graph()) 

In [62]:
# run it
epochs=1000
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for epoch in range(1, epochs):
    # loss, summary  = sess.run([train_op, summary_op], feed_dict)
    feed_dict = {
        X: X_train_scaled,
        y: y_train.reshape(-1,1),
        learning_rate: .002/epoch
    }
    # run
    _, summary = sess.run([train_op, summary_op], feed_dict)

    # log results
    writer.add_summary(summary)
    
    # log to console
    if epoch%(epochs/10) == 0:
        curr_loss, curr_W, curr_b, pred = sess.run([loss, W, b, y_pred], feed_dict)
        # print(curr_loss)
        curr_loss, pred = sess.run([loss, res], feed_dict)
        acc = accuracy_score(y_train, pred)
        print("Acc: {:.2f}, loss: {:.2f} ".format(acc, curr_loss))



Acc: 0.96, loss: 20.84 
Acc: 0.97, loss: 20.05 
Acc: 0.97, loss: 19.64 
Acc: 0.97, loss: 19.37 
Acc: 0.97, loss: 19.17 
Acc: 0.97, loss: 19.01 
Acc: 0.97, loss: 18.88 
Acc: 0.97, loss: 18.77 
Acc: 0.97, loss: 18.68 


In [63]:
np.bincount(pred.astype(np.int32).reshape(-1))
pred[:10], y_train[:10]

(array([[ 1.],
        [ 0.],
        [ 1.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 1.],
        [ 0.],
        [ 1.],
        [ 1.]], dtype=float32), array([1, 0, 1, 0, 0, 0, 1, 0, 1, 1]))

In [64]:
X_train[0], curr_W[:, 0], np.dot(X_train, curr_W)[0]

(array([  1.28900000e+01,   1.31200000e+01,   8.18900000e+01,
          5.15900000e+02,   6.95500000e-02,   3.72900000e-02,
          2.26000000e-02,   1.17100000e-02,   1.33700000e-01,
          5.58100000e-02,   1.53200000e-01,   4.69000000e-01,
          1.11500000e+00,   1.26800000e+01,   4.73100000e-03,
          1.34500000e-02,   1.65200000e-02,   5.90500000e-03,
          1.61900000e-02,   2.08100000e-03,   1.36200000e+01,
          1.55400000e+01,   8.74000000e+01,   5.77000000e+02,
          9.61600000e-02,   1.14700000e-01,   1.18600000e-01,
          5.36600000e-02,   2.30900000e-01,   6.91500000e-02]),
 array([-0.21686573, -0.17670448, -0.3008979 , -0.17692585, -0.06934768,
        -0.13862576, -0.26156288, -0.20703599, -0.08494963,  0.10557546,
        -0.17599268,  0.05136869, -0.12630957, -0.14625683,  0.01987111,
         0.03724913,  0.01506586, -0.03765457,  0.00518631,  0.03934653,
        -0.26747426, -0.21415108, -0.26601493, -0.27641252, -0.21696471,
        -0.08

In [65]:
feed_dict = {
    X: scaler.transform(X_train),
    y: y_train.reshape(-1,1)
}
curr_loss, pred = sess.run([loss, res], feed_dict)
acc = accuracy_score(y_train, pred)
print("Acc: {:.4f}, loss: {:.2f} ".format(acc, curr_loss))

feed_dict = {
    X: scaler.transform(X_test),
    y: y_test.reshape(-1,1)
}
curr_loss, pred = sess.run([loss, res], feed_dict)
acc = accuracy_score(y_test, pred)
print("Acc: {:.4f}, loss: {:.2f} ".format(acc, curr_loss))


Acc: 0.9671, loss: 18.59 
Acc: 0.9790, loss: 5.11 


# In scikit-learn

In [80]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [81]:
pred = clf.predict(X_train)
acc = accuracy_score(y_train, pred)
print("Acc: {:.4f}".format(acc))


Acc: 0.9695


In [82]:
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print("Acc: {:.4f}".format(acc))


Acc: 0.9580


In [83]:
X_train[0], clf.coef_, clf.intercept_

(array([  1.28900000e+01,   1.31200000e+01,   8.18900000e+01,
          5.15900000e+02,   6.95500000e-02,   3.72900000e-02,
          2.26000000e-02,   1.17100000e-02,   1.33700000e-01,
          5.58100000e-02,   1.53200000e-01,   4.69000000e-01,
          1.11500000e+00,   1.26800000e+01,   4.73100000e-03,
          1.34500000e-02,   1.65200000e-02,   5.90500000e-03,
          1.61900000e-02,   2.08100000e-03,   1.36200000e+01,
          1.55400000e+01,   8.74000000e+01,   5.77000000e+02,
          9.61600000e-02,   1.14700000e-01,   1.18600000e-01,
          5.36600000e-02,   2.30900000e-01,   6.91500000e-02]),
 array([[ 1.29403182,  0.18630806, -0.26510877,  0.00381304, -0.23469085,
         -0.31661328, -0.63018851, -0.38801023, -0.33400371, -0.05324722,
         -0.22329099,  1.98995163,  0.5314017 , -0.1216217 , -0.03877839,
          0.03169099, -0.04367506, -0.04633437, -0.0634173 ,  0.00541859,
         -0.3640177 , -0.38160754,  0.00858562,  0.00209213, -0.44857988,
        

In [84]:
list(zip(curr_W[:, 0], clf.coef_[0, :])), list(zip(curr_b, clf.intercept_))

([(-0.21686573, 1.2940318166436668),
  (-0.17670448, 0.18630805858555277),
  (-0.3008979, -0.26510876545417261),
  (-0.17692585, 0.0038130407408516476),
  (-0.06934768, -0.23469085315994104),
  (-0.13862576, -0.31661328459765403),
  (-0.26156288, -0.63018850629549483),
  (-0.20703599, -0.3880102334466336),
  (-0.084949628, -0.33400370586719041),
  (0.10557546, -0.053247224516927157),
  (-0.17599268, -0.22329098962735511),
  (0.051368695, 1.9899516330539277),
  (-0.12630957, 0.53140170028773603),
  (-0.14625683, -0.12162170000926409),
  (0.019871106, -0.038778394675246207),
  (0.037249129, 0.031690990788804205),
  (0.015065859, -0.043675063950918719),
  (-0.037654575, -0.046334366265279987),
  (0.0051863068, -0.063417298637916467),
  (0.039346531, 0.0054185856164800893),
  (-0.26747426, -0.3640177012525676),
  (-0.21415108, -0.3816075401627117),
  (-0.26601493, 0.0085856164562869708),
  (-0.27641252, 0.0020921327178484717),
  (-0.21696471, -0.44857988478368283),
  (-0.083709277, -0.9687